# Event Module

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob

import csv
import json

import math
import unittest

from common import testExit, projdir

from name import Name
from fuzzy import FuzzyMatch
from entrant import Entrant
from reports import Reports

from period import Period
from session import Session

from constants import *

In [2]:
EVENT_CONFIG = 'event.json'

## Event Class

Class to manage events

In [3]:
class Event(Period):
    def __init__(self, path, appConfig, existingNames={}, verbosity=1):
        
        super().__init__(verbosity=verbosity)

        self.path = path
        self.year = int(os.path.basename(path))

        self.appConfig = appConfig

        self.existingNames = existingNames
        
        self.sessions = {}
        
        self.initialised = False


    def checkName(self, names):
        '''Check if name is a potential typo'''

        for name in [name.strip() for name in names.split('+')]:
            if name in self.existingNames:
                self.existingNames[name].years.append(self.year)
            else:
                nameObj = Name(name)
                for existingName in self.existingNames:
                    existingNameObj = self.existingNames[existingName]
                    if not name.startswith('Sail Number') and self.fuzzyMatch.matchNameObjects(nameObj, existingNameObj) is True:
                        self.logWarning("Similar names - '{}' vs '{}' {}".format(
                            name, existingNameObj.name, existingNameObj.years))

                self.existingNames[name] = nameObj
                nameObj.years = [self.year]


    def loadConfig(self):
        '''Read app config from JSON'''

        filename = os.path.join(self.path, CONFIG_DIR, EVENT_CONFIG)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            try:
                self.eventConfig = json.loads(jsonTxt)
            except:
                 self.logError('Could not parse {}'.format(filename))


    def loadEntrants(self):
        '''Read entrants from JSON'''

        csvPath = os.path.join(self.path, CONFIG_DIR, ENTRANTS_CSV)
        
        with open(csvPath, 'r', encoding='utf-8') as f:
            csvReader = csv.reader(f)
            headers = next(csvReader)

            for values in csvReader:
                entrant = Entrant(headers, values, verbosity=self.verbosity)
                if entrant.getValue('ID') not in self.entrants:
                    self.entrants[entrant.getValue('ID')] = entrant
                    
                    self.checkName(entrant.getValue('Name'))
                else:
                    raise ValueError('Duplicate entrant ID "{}"'.format(entrant.getValue('ID')))

        self.logInfo('{} entrants loaded'.format(len(self.entrants)))


    def loadReports(self):
        '''Read reports from JSON and (indirectly) pre-apply the entrant filters'''

        if 'Decimals' in self.eventConfig:
            decimals = self.eventConfig['Decimals']
        else:
            decimals = 2

        self.reports = Reports(self.path, self.entrants, decimals=decimals, verbosity=self.verbosity)

        self.reports.loadReports()
        

    def indexNames(self):
        '''Create name indices for entrants'''

        for entrantId in self.entrants:
            entrant = self.entrants[entrantId]
            name = entrant.getName()

            if name in self.names:
                self.names[name].append(entrant)
            else:
                self.names[name] = [entrant]
        

    def indexSails(self):
        '''Create sail number indices for entrants'''

        for entrantId in self.entrants:
            entrant = self.entrants[entrantId]

            for sailNo in entrant.sailNos:
                if sailNo.upper() not in self.sailNos:
                    self.sailNos[sailNo.upper()] = entrant
                else:
                    raise ValueError('Duplicate sail number \"{}\"'.format(sailNo))
        

    def indexGt31s(self):
        '''Create GT-31 indices for entrants'''

        for entrantId in self.entrants:
            entrant = self.entrants[entrantId]

            for gt31Id in entrant.gt31Ids:
                if gt31Id.upper() not in self.gt31Ids:
                    self.gt31Ids[gt31Id.upper()] = entrant
                else:
                    raise ValueError('Duplicate GT-31 ID \"{}\"'.format(gt31Id))
        

    def loadSession(self, configPath, dataPath, dataType):
        '''Load a single event session - absence of config will result in an exception (intentional)'''
        
        session = Session(self, dataPath, configPath, verbosity=self.verbosity)

        session.loadRuns(dataType)

        self.sessions[session.date] = session


    def loadSessions(self):
        '''Load all of the event sessions'''
        
        for dataDir in [RUNDATA_DIR, GPSDATA_DIR]:
            dataPaths = sorted(glob.glob(os.path.join(self.path, dataDir, '[1-2][0-9][0-9][0-9][0-1][0-9][0-3][0-9]')))   
            for dataPath in dataPaths:
                configPath = os.path.join(self.path, SESSIONS_DIR, os.path.basename(dataPath))
                self.loadSession(configPath, dataPath, dataDir)

        self.sortRuns()


    def initEvent(self):
        '''Load entrants and generate all of the indices'''

        self.loadConfig()
        self.loadEntrants()
        self.loadReports()
        self.indexNames()
        self.indexSails()
        self.indexGt31s()
        
        self.initialised = True
        

    def processEvent(self):
        '''Read entrants from config folder'''

        if self.verbosity >= 1:
            print('Processing {}...'.format(self.year))

        if not self.initialised:
            self.initEvent()
        self.loadSessions()

        for sessionId in self.sessions:
            session = self.sessions[sessionId]
            session.runReports()
        self.runReports()

        if self.verbosity >= 1:
            print('All done!\n')


    def loadResults(self):
        '''Load all of the published results'''
        
        for dataDir in [RESULTS_DIR]:
            dataPaths = sorted(glob.glob(os.path.join(self.path, dataDir, '[1-2][0-9][0-9][0-9][0-1][0-9][0-3][0-9]')))   
            for dataPath in dataPaths:
                configPath = os.path.join(self.path, SESSIONS_DIR, os.path.basename(dataPath))
                self.loadSession(configPath, dataPath, dataDir)

        self.sortRuns()

## Results Check

Standalone function to check that calculated results match published results

In [4]:
def compareResults(actualSession, expectedSession, actualResult, expectedResult):
    '''Compare expected vs actual results for a single session'''

    actualSpeed = actualResult.data[T_SPEED]
    expectedSpeed = expectedResult.data[T_SPEED]

    if actualSpeed != expectedSpeed:
        if int(actualSession.year) >= 2000 and int(actualSession.year) <= 2004:
            # Handle minor rounding errors in historical results, taking account of floating point artefacts
            if actualSpeed != math.trunc(round((expectedSpeed + 0.01) * 1000) / 10) / 100:
                actualSession.logError('Different session results for {} on {} - {} vs {}'.format(
                    actualResult.entrant.getName(), actualSession.date, actualSpeed, expectedSpeed))
        else:
            actualSession.logError('Different session results for {} on {} - {} vs {}'.format(
                actualResult.entrant.getName(), actualSession.date, actualSpeed, expectedSpeed))

    # No need to check names on the actual results because that is done when they are loaded into memory
    if actualResult.entrant.getName() != expectedResult.entrant.getName():
        actualSession.logError('Name mismatch on {} - {} vs {}'.format(
            actualSession.date, actualResult.entrant.getName(), expectedResult.entrant.getName()))

    # Run times are only present on session results for 2006, 2007 and 2009
    for timeField in ['Time', 'Start Time', 'Finish Time']:
        if timeField in actualResult.data and timeField in expectedResult.data:
            actualTime = actualResult.data[timeField]
            expectedTime = actualResult.data[timeField]
            if actualTime != expectedTime:
                actualSession.logError('Different {} for {} on {} - {} vs {}'.format(
                    timeField.lower(), actualResult.entrant.getName(), actualSession.date, actualTime, expectedTime))

    if 'Craft Type' in actualResult.data and actualResult.data['Craft Type']:
        resultCraftType = actualResult.data['Craft Type'].replace(' ', '').lower()
        entrantCraftType = actualResult.entrant.getValue('Craft Type').replace(' ', '').lower()
        if resultCraftType != entrantCraftType:
            actualSession.logError('Craft type mismatch for {} on {} - {} vs {}'.format(
                actualResult.entrant.getName(), actualSession.date,
                actualResult.data['Craft Type'], actualResult.entrant.getValue('Craft Type')))

    if 'Craft Type' in expectedResult.data and expectedResult.data['Craft Type']:
        resultCraftType = expectedResult.data['Craft Type'].replace(' ', '').lower()
        entrantCraftType = expectedResult.entrant.getValue('Craft Type').replace(' ', '').lower()
        if resultCraftType != entrantCraftType:
            actualSession.logError('Craft type mismatch for {} on {} - {} vs {}'.format(
                expectedResult.entrant.getName(), expectedSession.date,
                expectedResult.data['Craft Type'], expectedResult.entrant.getValue('Craft Type')))


def compareSessions(actualSession, expectedSession):
    '''Compare expected vs actual results for a single session'''

    if len(actualSession.runs) != len(expectedSession.runs):
        expectedSession.logError('Number of entrants does not match on {} - {} (actual) vs {} (expected)'.format(
            expectedSession.date, len(actualSession.runs), len(expectedSession.runs)))

    for entrantId in expectedSession.runs:
        if entrantId not in actualSession.runs:
            expectedSession.logWarning('Missing result for {} on {} - {} knots'.format(
                expectedSession.entrants[entrantId].getName(), expectedSession.date,
                expectedSession.runs[entrantId][0].data[T_SPEED]))           

    for entrantId in actualSession.runs:
        if entrantId not in expectedSession.runs:
            actualSession.logError('Unexpected result for {} on {} - {} knots'.format(
                actualSession.entrants[entrantId].getName(), actualSession.date,
                actualSession.runs[entrantId][0].data[T_SPEED]))           
        else:
            compareResults(actualSession, expectedSession, 
                           actualSession.runs[entrantId][0], expectedSession.runs[entrantId][0])


def testResults():
    '''Test event results match what was published on the website at the time'''
    
    verbosity = 1

    for eventYear in range(1998, 2010):
        if verbosity >= 1:
            print('Testing {}...'.format(eventYear))

        eventPath = os.path.join(projdir, EVENTS_DIR, str(eventYear))
        
        actual = Event(eventPath, appConfig)
        actual.initEvent()
        actual.loadSessions()

        expected = Event(eventPath, appConfig)
        expected.initEvent()
        expected.loadResults()
        
        for sessionDate in expected.sessions:
            actualSession = actual.sessions[sessionDate]
            expectedSession = expected.sessions[sessionDate]
            compareSessions(actualSession, expectedSession)

        if verbosity >= 1:
            print('All done!\n')


def testReports():
    '''Test event results match what was published on the website at the time'''

    verbosity = 1

    for eventYear in range(1998, 1999):
        if verbosity >= 1:
            print('Reporting {}...'.format(eventYear))

        eventPath = os.path.join(projdir, EVENTS_DIR, str(eventYear))
        
        event = Event(eventPath, appConfig)
        event.initEvent()
        event.loadSessions()
        
        for sessionId in event.sessions:
            session = event.sessions[sessionId]
            session.runReports()
        event.runReports()

        if verbosity >= 1:
            print('All done!\n')

## Unit Tests

A handful of very basic tests

In [5]:
class TestEvent(unittest.TestCase):
    '''Class to test Event class'''
    
    def testLoadEntrants(self):
        '''Test using 2003 data'''

        eventYear = '2003'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        event = Event(eventPath, appConfig, verbosity=0)

        event.loadEntrants()
        
        # Check the entrants = 35 in entrants.csv
        self.assertEqual(len(event.entrants), 35)


    def testIndexNames(self):
        '''Test using 2003 data'''

        eventYear = '2003'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        event = Event(eventPath, appConfig)

        event.loadEntrants()
        event.indexNames()
        
        # Check the names = 34 in entrants.csv (2 craft types for Richard Trubger)
        self.assertEqual(len(event.names), 34)


    def testIndexSails(self):
        '''Test using 2003 data'''

        eventYear = '2003'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        event = Event(eventPath, appConfig)

        event.loadEntrants()
        event.indexSails()
        
        # Check the sail numbers = 36 in entrants.csv (2 sail numbers for Torix)
        self.assertEqual(len(event.sailNos), 36)


    def testIndexGt31Ids(self):
        '''Test using 2019 data'''

        eventYear = '2019'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        event = Event(eventPath, appConfig)

        event.loadEntrants()
        event.indexGt31s()
        
        # Check the entrants = 68 in entrants.csv
        self.assertEqual(len(event.gt31Ids), 72)


    def testProcessEvent(self):
        '''Test using 2019 data'''

        eventYear = '2019'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)

        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        event = Event(eventPath, appConfig, verbosity=0)

        event.processEvent()
        
        # Check the entrants = 68 in entrants.csv and 4 unrecognised GPS
        self.assertEqual(len(event.entrants), 68 + 4)

        # Check the event totals
        self.assertEqual(event.numRuns, 3731)
        self.assertEqual(len(event.runs), 68)
        
        # Check runs are sorted correctly
        for personId in event.runs:
            maxSpeed = 99.999
            for run in event.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [6]:
if __name__ == '__main__':
    # Read main config into global variable
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        appConfig = json.loads(jsonTxt)
    
    testResults()
    #testReports()

Testing 1998...
All done!

Testing 1999...
All done!

Testing 2000...
All done!

Testing 2001...
All done!

Testing 2002...
All done!

Testing 2003...
All done!

Testing 2004...
All done!

Testing 2005...
All done!

Testing 2006...
All done!

Testing 2007...
All done!

Testing 2008...
All done!

Testing 2009...
All done!



In [7]:
if __name__ == '__main__':
       unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.238s

OK


## All Done!